# Módulo 3: Deep Learning para Visão Computacional## Objetivos de Aprendizagem- Compreender arquiteturas de CNNs (Redes Neurais Convolucionais)- Entender o funcionamento de redes neurais convolucionais- Conhecer arquiteturas clássicas e sua evolução- Analisar diferenças entre AlexNet, VGG e ResNet- Implementar conceitos práticos com PyTorch---## 3.1 Redes Neurais Convolucionais (CNNs)As **Redes Neurais Convolucionais** são arquiteturas de deep learning especificamente projetadas para processar dados com estrutura espacial, como imagens. Elas foram inspiradas no sistema visual biológico e revolucionaram a visão computacional.![CNNs - Conceito Geral](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo3/cnns_conceito_geral.png)### Arquitetura Básica de CNNs**Componentes Fundamentais:**![Arquitetura Básica CNNs](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo3/arquitetura_basica_cnns.png)#### **1. Camadas Convolucionais**- **Filtros/Kernels aprendíveis**: Matrizes de pesos que detectam características específicas- **Campo receptivo local**: Cada neurônio conecta-se apenas a uma região local da entrada- **Compartilhamento de parâmetros**: Mesmo filtro aplicado em toda a imagem- **Operação de convolução matemática**: Produto elemento a elemento + soma- **Feature maps**: Mapas de características extraídas em cada camada- **Detecção hierárquica**: Padrões simples → complexos conforme profundidade**Matemática da Convolução:**```Y[i,j] = Σ Σ X[i+m, j+n] * W[m,n] + b```#### **2. Camadas de Pooling**- **Max Pooling**: Seleciona o valor máximo em cada região- **Average Pooling**: Calcula a média dos valores na região- **Redução de dimensionalidade espacial**: Diminui tamanho das feature maps- **Invariância a translação**: Robustez a pequenos deslocamentos- **Stride**: Passo de deslizamento do kernel- **Receptive field**: Campo de visão de cada neurônio#### **3. Camadas Fully Connected**- **Classificação final**: Conecta todas as features extraídas- **Função de ativação**: ReLU, Sigmoid, Softmax- **Dropout**: Regularização para prevenir overfitting- **Output**: Probabilidades para cada classe### Evolução das Arquiteturas**Progressão Histórica:**- **2012 - AlexNet**: Primeira CNN vencedora do ImageNet- **2014 - VGG**: Arquitetura simples e consistente- **2015 - ResNet**: Skip connections e redes profundas- **2016+**: Evoluções e melhorias incrementais![Evolução das Arquiteturas](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo3/evolucao_arquiteturas.png)**Tendências Observadas:**- **Profundidade**: Aumento constante do número de camadas- **Eficiência**: Redução de parâmetros com melhor performance- **Inovação**: Resolução de problemas específicos- **Aplicação**: Transfer learning e modelos pré-treinados**Referências:**- [Deep Learning - Goodfellow, Bengio & Courville](https://www.deeplearningbook.org/)- [CS231n Course - Stanford](http://cs231n.stanford.edu/)

## 3.2 Demonstração Prática: Visualização de CNNs

Vamos implementar e visualizar como as CNNs processam imagens:


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import cv2
from torchvision import transforms
from PIL import Image

class SimpleCNN(nn.Module):
    """CNN simples para demonstração"""
    
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        # Camada convolucional 1
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2)
        
        # Pooling
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Camada convolucional 2
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=2)
        
        # Camadas fully connected
        self.fc1 = nn.Linear(16 * 8 * 8, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Primeira camada convolucional + pooling
        x = self.pool(F.relu(self.conv1(x)))
        
        # Segunda camada convolucional + pooling
        x = self.pool(F.relu(self.conv2(x)))
        
        # Flatten para fully connected
        x = x.view(-1, 16 * 8 * 8)
        
        # Camadas fully connected
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x

def create_sample_image():
    """Cria uma imagem de exemplo para demonstração"""
    # Criar imagem sintética com formas
    img = np.zeros((32, 32), dtype=np.uint8)
    
    # Adicionar formas
    cv2.circle(img, (16, 16), 8, 255, -1)
    cv2.rectangle(img, (8, 8), (24, 24), 128, 2)
    
    # Adicionar ruído
    noise = np.random.normal(0, 10, img.shape).astype(np.uint8)
    img = cv2.add(img, noise)
    
    return img

def visualize_cnn_layers():
    """Visualiza como uma CNN processa uma imagem"""
    
    # Criar modelo
    model = SimpleCNN()
    model.eval()
    
    # Criar imagem de exemplo
    img = create_sample_image()
    
    # Converter para tensor
    img_tensor = torch.FloatTensor(img).unsqueeze(0).unsqueeze(0) / 255.0
    
    # Processar através das camadas
    with torch.no_grad():
        # Primeira camada convolucional
        conv1_out = F.relu(model.conv1(img_tensor))
        pool1_out = model.pool(conv1_out)
        
        # Segunda camada convolucional
        conv2_out = F.relu(model.conv2(pool1_out))
        pool2_out = model.pool(conv2_out)
        
        # Flatten
        flattened = pool2_out.view(-1, 16 * 8 * 8)
        
        # Fully connected
        fc1_out = F.relu(model.fc1(flattened))
        fc2_out = F.relu(model.fc2(fc1_out))
        output = model.fc3(fc2_out)
        
    # Visualização
    fig, axes = plt.subplots(3, 6, figsize=(18, 12))
    
    # Imagem original
    axes[0, 0].imshow(img, cmap='gray')
    axes[0, 0].set_title('Imagem Original\n(32×32)')
    axes[0, 0].axis('off')
    
    # Primeira camada convolucional (6 filtros)
    for i in range(6):
        if i < 5:
            axes[0, i+1].imshow(conv1_out[0, i].numpy(), cmap='viridis')
            axes[0, i+1].set_title(f'Conv1 - Filtro {i+1}')
            axes[0, i+1].axis('off')
        else:
            axes[0, i+1].axis('off')
    
    # Primeira camada de pooling
    for i in range(6):
        if i < 5:
            axes[1, i+1].imshow(pool1_out[0, i].numpy(), cmap='viridis')
            axes[1, i+1].set_title(f'Pool1 - Filtro {i+1}')
            axes[1, i+1].axis('off')
        else:
            axes[1, i+1].axis('off')
    
    # Segunda camada convolucional (16 filtros)
    for i in range(6):
        if i < 5:
            axes[2, i+1].imshow(conv2_out[0, i].numpy(), cmap='viridis')
            axes[2, i+1].set_title(f'Conv2 - Filtro {i+1}')
            axes[2, i+1].axis('off')
        else:
            axes[2, i+1].axis('off')
    
    # Adicionar informações sobre dimensões
    axes[1, 0].text(0.5, 0.5, f'Conv1 Output:\n{conv1_out.shape[2]}×{conv1_out.shape[3]}\n6 filtros', 
                    ha='center', va='center', fontsize=10,
                    bbox=dict(boxstyle='round', facecolor='lightblue'))
    axes[1, 0].axis('off')
    
    axes[2, 0].text(0.5, 0.5, f'Conv2 Output:\n{conv2_out.shape[2]}×{conv2_out.shape[3]}\n16 filtros', 
                    ha='center', va='center', fontsize=10,
                    bbox=dict(boxstyle='round', facecolor='lightgreen'))
    axes[2, 0].axis('off')
    
    plt.suptitle('Processamento de Imagem através de CNN', fontsize=16)
    plt.tight_layout()
    plt.show()
    
    return {
        'original': img,
        'conv1': conv1_out,
        'pool1': pool1_out,
        'conv2': conv2_out,
        'pool2': pool2_out,
        'output': output
    }

# Executar demonstração
cnn_results = visualize_cnn_layers()

### Análise dos Resultados

**Processamento Observado:**

1. **Imagem Original (32×32)**: Formas geométricas com ruído
2. **Conv1 (6 filtros)**: Detecção de bordas e padrões básicos
3. **Pool1**: Redução de dimensionalidade, preservação de características
4. **Conv2 (16 filtros)**: Detecção de padrões mais complexos
5. **Pool2**: Redução final, preparação para classificação

**Insights Importantes:**
- **Hierarquia**: Padrões simples → complexos
- **Redução**: Dimensionalidade diminui a cada camada
- **Features**: Cada filtro detecta características específicas
- **Robustez**: Pooling aumenta invariância a translação

**Referências:**
- [Deep Learning - Goodfellow, Bengio & Courville](https://www.deeplearningbook.org/)


## 3.3 Arquiteturas Clássicas### AlexNet (2012) - A Revolução**Características Principais:**- **8 camadas**: 5 convolucionais + 3 fully connected- **60M parâmetros**: Modelo grande para época- **ReLU activation**: Primeira aplicação em larga escala- **Dropout**: Regularização eficaz- **Data augmentation**: Expansão do dataset- **GPU training**: Paralelização massiva![Arquitetura AlexNet](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo3/arquitetura_alexnet.png)**Inovações Técnicas:**- **ReLU**: Solução para vanishing gradient- **Dropout**: Prevenção de overfitting- **GPU**: Treinamento em paralelo- **Data augmentation**: Rotação, espelhamento, crop### VGG (2014) - Simplicidade e Profundidade**Características Principais:**- **16-19 camadas**: VGG-16 e VGG-19- **138M parâmetros**: Modelo muito grande- **Kernels 3×3**: Filtros pequenos e consistentes- **Arquitetura simples**: Padrão repetitivo- **Transfer learning**: Base para muitos modelos![Arquitetura VGG](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo3/arquitetura_vgg.png)**Vantagens:**- **Simplicidade**: Arquitetura fácil de entender- **Profundidade**: Muitas camadas para features complexas- **Transfer learning**: Excelente para fine-tuning- **Robustez**: Performance consistente### ResNet (2015) - Skip Connections**Características Principais:**- **50-152 camadas**: ResNet-50, ResNet-101, ResNet-152- **25M parâmetros**: Menos que VGG-16- **Skip connections**: Conexões residuais- **Batch normalization**: Normalização por lotes- **Degradação resolvida**: Treinamento de redes muito profundas![Arquitetura ResNet](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo3/arquitetura_resnet.png)**Inovações Técnicas:**- **Skip connections**: H(x) = F(x) + x- **Residual learning**: Aprende diferenças (resíduos)- **Batch normalization**: Estabilização do treinamento- **Identity mapping**: Preserva informação original**Referências:**- [ImageNet Classification with Deep Convolutional Neural Networks - Krizhevsky et al.](https://papers.nips.cc/paper/2012/hash/c399862d3b9d6b76c8436e924a68c45b-Abstract.html)- [Very Deep Convolutional Networks for Large-Scale Image Recognition - Simonyan & Zisserman](https://arxiv.org/abs/1409.1556)- [Deep Residual Learning for Image Recognition - He et al.](https://arxiv.org/abs/1512.03385)

## 3.4 Demonstração Prática: Comparação de Arquiteturas

Vamos implementar e comparar diferentes arquiteturas de CNN:


In [ ]:
import torch.nn as nn
import torch
import matplotlib.pyplot as plt
import numpy as np

class AlexNetLike(nn.Module):
    """Implementação simplificada do AlexNet"""
    
    def __init__(self, num_classes=10):
        super(AlexNetLike, self).__init__()
        
        # Camadas convolucionais
        self.conv1 = nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2)
        self.conv2 = nn.Conv2d(64, 192, kernel_size=5, padding=2)
        self.conv3 = nn.Conv2d(192, 384, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(384, 256, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        
        # Pooling
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2)
        
        # Dropout
        self.dropout = nn.Dropout(0.5)
        
        # Fully connected
        self.fc1 = nn.Linear(256 * 6 * 6, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, num_classes)
        
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = torch.relu(self.conv3(x))
        x = torch.relu(self.conv4(x))
        x = self.pool(torch.relu(self.conv5(x)))
        
        x = x.view(x.size(0), -1)
        x = self.dropout(torch.relu(self.fc1(x)))
        x = self.dropout(torch.relu(self.fc2(x)))
        x = self.fc3(x)
        
        return x

class VGGLike(nn.Module):
    """Implementação simplificada do VGG"""
    
    def __init__(self, num_classes=10):
        super(VGGLike, self).__init__()
        
        # Camadas convolucionais (VGG-16 simplificado)
        self.features = nn.Sequential(
            # Bloco 1
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Bloco 2
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Bloco 3
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        
        # Classificador
        self.classifier = nn.Sequential(
            nn.Linear(256 * 4 * 4, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes),
        )
        
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

class ResNetLike(nn.Module):
    """Implementação simplificada do ResNet"""
    
    def __init__(self, num_classes=10):
        super(ResNetLike, self).__init__()
        
        # Camada inicial
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        # Blocos residuais
        self.layer1 = self._make_layer(64, 64, 2)
        self.layer2 = self._make_layer(64, 128, 2, stride=2)
        self.layer3 = self._make_layer(128, 256, 2, stride=2)
        
        # Classificador
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(256, num_classes)
        
    def _make_layer(self, in_channels, out_channels, blocks, stride=1):
        layers = []
        
        # Primeiro bloco com stride
        layers.append(BasicBlock(in_channels, out_channels, stride))
        
        # Blocos restantes
        for _ in range(1, blocks):
            layers.append(BasicBlock(out_channels, out_channels))
        
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.maxpool(torch.relu(self.bn1(self.conv1(x))))
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

class BasicBlock(nn.Module):
    """Bloco básico do ResNet"""
    
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        # Skip connection
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )
        
    def forward(self, x):
        residual = x
        
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        
        out += self.shortcut(residual)
        out = torch.relu(out)
        
        return out

def compare_architectures():
    """Compara diferentes arquiteturas de CNN"""
    
    # Criar modelos
    alexnet = AlexNetLike()
    vgg = VGGLike()
    resnet = ResNetLike()
    
    # Calcular parâmetros
    alexnet_params = sum(p.numel() for p in alexnet.parameters())
    vgg_params = sum(p.numel() for p in vgg.parameters())
    resnet_params = sum(p.numel() for p in resnet.parameters())
    
    # Criar dados de exemplo
    input_tensor = torch.randn(1, 3, 224, 224)
    
    # Testar forward pass
    with torch.no_grad():
        alexnet.eval()
        vgg.eval()
        resnet.eval()
        
        alexnet_output = alexnet(input_tensor)
        vgg_output = vgg(input_tensor)
        resnet_output = resnet(input_tensor)
    
    # Visualização
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # Informações dos modelos
    models_info = [
        ('AlexNet', alexnet_params, alexnet_output.shape),
        ('VGG', vgg_params, vgg_output.shape),
        ('ResNet', resnet_params, resnet_output.shape)
    ]
    
    # Gráfico de parâmetros
    model_names = [info[0] for info in models_info]
    param_counts = [info[1] for info in models_info]
    
    axes[0, 0].bar(model_names, param_counts, color=['red', 'green', 'blue'])
    axes[0, 0].set_title('Número de Parâmetros')
    axes[0, 0].set_ylabel('Parâmetros (milhões)')
    axes[0, 0].tick_params(axis='x', rotation=45)
    
    # Adicionar valores nas barras
    for i, v in enumerate(param_counts):
        axes[0, 0].text(i, v + max(param_counts) * 0.01, f'{v/1e6:.1f}M', 
                        ha='center', va='bottom')
    
    # Comparação de características
    characteristics = {
        'Profundidade': [8, 16, 18],
        'Inovação': [3, 2, 5],  # Escala 1-5
        'Simplicidade': [3, 5, 2],  # Escala 1-5
        'Performance': [3, 4, 5]  # Escala 1-5
    }
    
    x = np.arange(len(model_names))
    width = 0.2
    
    for i, (char, values) in enumerate(characteristics.items()):
        axes[0, 1].bar(x + i*width, values, width, label=char)
    
    axes[0, 1].set_title('Comparação de Características')
    axes[0, 1].set_ylabel('Pontuação')
    axes[0, 1].set_xticks(x + width * 1.5)
    axes[0, 1].set_xticklabels(model_names)
    axes[0, 1].legend()
    
    # Evolução temporal
    years = [2012, 2014, 2015]
    accuracy = [15.3, 7.3, 3.6]  # Top-5 error rates
    
    axes[0, 2].plot(years, accuracy, 'o-', linewidth=2, markersize=8)
    axes[0, 2].set_title('Evolução da Performance (ImageNet)')
    axes[0, 2].set_xlabel('Ano')
    axes[0, 2].set_ylabel('Top-5 Error Rate (%)')
    axes[0, 2].grid(True, alpha=0.3)
    
    # Adicionar labels dos pontos
    for i, (year, acc) in enumerate(zip(years, accuracy)):
        axes[0, 2].annotate(f'{model_names[i]}\n{acc}%', 
                          (year, acc), textcoords="offset points", 
                          xytext=(0,10), ha='center')
    
    # Informações detalhadas
    info_text = """
    ALEXNET (2012):
    • Primeira CNN vencedora do ImageNet
    • ReLU activation
    • Dropout regularization
    • GPU training
    
    VGG (2014):
    • Arquitetura simples e consistente
    • Kernels 3×3 apenas
    • Muito profundo
    • Excelente para transfer learning
    
    RESNET (2015):
    • Skip connections
    • Resolve problema de degradação
    • Batch normalization
    • Redes muito profundas
    """
    
    axes[1, 0].text(0.05, 0.95, info_text, transform=axes[1, 0].transAxes, 
                    fontsize=10, verticalalignment='top',
                    bbox=dict(boxstyle='round', facecolor='lightgray', alpha=0.8))
    axes[1, 0].set_title('Características Principais')
    axes[1, 0].axis('off')
    
    # Vantagens e desvantagens
    comparison_text = """
    COMPARAÇÃO:
    
    ALEXNET:
    ✓ Pioneiro, ReLU, Dropout
    ✗ Pouco profundo, muitos parâmetros
    
    VGG:
    ✓ Simples, consistente, transfer learning
    ✗ Muitos parâmetros, lento
    
    RESNET:
    ✓ Skip connections, eficiente, profundo
    ✗ Complexo, difícil de interpretar
    """
    
    axes[1, 1].text(0.05, 0.95, comparison_text, transform=axes[1, 1].transAxes, 
                    fontsize=10, verticalalignment='top',
                    bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8))
    axes[1, 1].set_title('Vantagens e Desvantagens')
    axes[1, 1].axis('off')
    
    # Impacto histórico
    impact_text = """
    IMPACTO HISTÓRICO:
    
    2012 - ALEXNET:
    • Início da era deep learning
    • Redução de erro de 26% para 16%
    • Revolução na visão computacional
    
    2014 - VGG:
    • Padrão de arquitetura
    • Base para muitos modelos
    • Transfer learning popularizado
    
    2015 - RESNET:
    • Resolve degradação
    • Redes de 100+ camadas
    • Influência em todas as áreas
    """
    
    axes[1, 2].text(0.05, 0.95, impact_text, transform=axes[1, 2].transAxes, 
                    fontsize=10, verticalalignment='top',
                    bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.8))
    axes[1, 2].set_title('Impacto Histórico')
    axes[1, 2].axis('off')
    
    plt.suptitle('Comparação de Arquiteturas Clássicas de CNN', fontsize=16)
    plt.tight_layout()
    plt.show()
    
    return {
        'alexnet_params': alexnet_params,
        'vgg_params': vgg_params,
        'resnet_params': resnet_params,
        'models_info': models_info
    }

# Executar comparação
comparison_results = compare_architectures()

### Análise da Comparação

**Características Observadas:**

1. **AlexNet (2012)**:
   - **Parâmetros**: ~60M (mais que ResNet)
   - **Profundidade**: 8 camadas
   - **Inovação**: Pioneiro em ReLU e Dropout
   - **Impacto**: Revolução na área

2. **VGG (2014)**:
   - **Parâmetros**: ~138M (mais que todos)
   - **Profundidade**: 16-19 camadas
   - **Simplicidade**: Arquitetura consistente
   - **Uso**: Excelente para transfer learning

3. **ResNet (2015)**:
   - **Parâmetros**: ~25M (mais eficiente)
   - **Profundidade**: 50-152 camadas
   - **Inovação**: Skip connections
   - **Performance**: Melhor accuracy

**Insights Importantes:**
- **Evolução**: Menos parâmetros, mais profundidade
- **Eficiência**: ResNet é mais eficiente que VGG
- **Inovação**: Cada arquitetura resolve problemas específicos
- **Impacto**: Progressão constante da performance

**Referências:**
- [ImageNet Classification with Deep Convolutional Neural Networks - Krizhevsky et al.](https://papers.nips.cc/paper/2012/hash/c399862d3b9d6b76c8436e924a68c45b-Abstract.html)
- [Very Deep Convolutional Networks for Large-Scale Image Recognition - Simonyan & Zisserman](https://arxiv.org/abs/1409.1556)
- [Deep Residual Learning for Image Recognition - He et al.](https://arxiv.org/abs/1512.03385)


## 3.5 Funções de Ativação e Regularização### Funções de Ativação**ReLU (Rectified Linear Unit):**- **Fórmula**: f(x) = max(0, x)- **Vantagens**: Simples, eficiente, resolve vanishing gradient- **Desvantagens**: Dying ReLU problem- **Uso**: Padrão em CNNs modernas**Leaky ReLU:**- **Fórmula**: f(x) = max(0.01x, x)- **Vantagens**: Evita dying ReLU- **Uso**: Alternativa ao ReLU**Sigmoid:**- **Fórmula**: f(x) = 1 / (1 + e^(-x))- **Uso**: Classificação binária- **Problema**: Vanishing gradient**Softmax:**- **Fórmula**: f(x_i) = e^(x_i) / Σ e^(x_j)- **Uso**: Classificação multiclasse- **Output**: Probabilidades normalizadas![Funções de Ativação](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo3/funcoes_ativacao.png)### Técnicas de Regularização**Dropout:**- **Funcionamento**: Desativa neurônios aleatoriamente- **Taxa**: 0.5 comum para fully connected- **Efeito**: Previne overfitting- **Uso**: Durante treinamento apenas**Batch Normalization:**- **Funcionamento**: Normaliza inputs por lote- **Efeito**: Acelera treinamento, estabiliza- **Uso**: Após camadas convolucionais- **Vantagem**: Permite learning rates maiores**Data Augmentation:**- **Técnicas**: Rotação, espelhamento, crop, color jitter- **Efeito**: Aumenta diversidade do dataset- **Uso**: Durante treinamento- **Vantagem**: Melhora generalização![Técnicas de Regularização](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo3/tecnicas_regularizacao.png)**Referências:**- [Deep Learning - Goodfellow, Bengio & Courville](https://www.deeplearningbook.org/)- [Batch Normalization: Accelerating Deep Network Training - Ioffe & Szegedy](https://arxiv.org/abs/1502.03167)

## Resumo do Módulo 3

### Principais Conceitos Abordados

1. **Redes Neurais Convolucionais (CNNs)**
   - Arquitetura básica e componentes
   - Camadas convolucionais, pooling e fully connected
   - Matemática da convolução
   - Processamento hierárquico de features

2. **Arquiteturas Clássicas**
   - AlexNet: Pioneiro e revolucionário
   - VGG: Simplicidade e profundidade
   - ResNet: Skip connections e eficiência
   - Evolução e comparação

3. **Funções de Ativação e Regularização**
   - ReLU, Leaky ReLU, Sigmoid, Softmax
   - Dropout, Batch Normalization, Data Augmentation
   - Técnicas para prevenir overfitting

### Demonstrações Práticas

**1. Visualização de CNNs:**
   - Processamento de imagem através de camadas
   - Feature maps em diferentes níveis
   - Análise de dimensionalidade

**2. Comparação de Arquiteturas:**
   - Implementação de AlexNet, VGG e ResNet
   - Análise de parâmetros e eficiência
   - Comparação de características

### Próximos Passos

No próximo módulo, exploraremos **Transfer Learning**, onde aprenderemos a usar modelos pré-treinados para resolver problemas específicos.

### Referências Principais

- [Deep Learning - Goodfellow, Bengio & Courville](https://www.deeplearningbook.org/)
- [ImageNet Classification with Deep Convolutional Neural Networks - Krizhevsky et al.](https://papers.nips.cc/paper/2012/hash/c399862d3b9d6b76c8436e924a68c45b-Abstract.html)
- [Deep Residual Learning for Image Recognition - He et al.](https://arxiv.org/abs/1512.03385)

---

**Próximo Módulo**: Transfer Learning e Aplicações Práticas
